<a href="https://colab.research.google.com/github/RajanMehta/mini-projects/blob/master/TextRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Document Summarization using TextRank**

In [0]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import CountVectorizer

document = """To Sherlock Holmes she is always the woman. I have
seldom heard him mention her under any other name. In his eyes she
eclipses and predominates the whole of her sex. It was not that he
felt any emotion akin to love for Irene Adler. All emotions, and that
one particularly, were abhorrent to his cold, precise but admirably
balanced mind. He was, I take it, the most perfect reasoning and
observing machine that the world has seen, but as a lover he would
have placed himself in a false position. He never spoke of the softer
passions, save with a gibe and a sneer. They were admirable things for
the observer-excellent for drawing the veil from men’s motives and
actions. But for the trained reasoner to admit such intrusions into
his own delicate and finely adjusted temperament was to introduce a
distracting factor which might throw a doubt upon all his mental
results. Grit in a sensitive instrument, or a crack in one of his own
high-power lenses, would not be more disturbing than a strong emotion
in a nature such as his. And yet there was but one woman to him, and
that woman was the late Irene Adler, of dubious and questionable
memory.
"""

# sentence splitting
document = ' '.join(document.strip().split('\n'))
sentence_tokenizer = PunktSentenceTokenizer()
sentences = sentence_tokenizer.tokenize(document)

# bag of words
c = CountVectorizer()
bow_matrix = c.fit_transform(sentences)
bow_matrix

<11x127 sparse matrix of type '<class 'numpy.int64'>'
	with 183 stored elements in Compressed Sparse Row format>

In [0]:
# normalize matrix. This will re-weight each word based upon its tf-idf, which will diminish the effect of words common to each sentence.
from sklearn.feature_extraction.text import TfidfTransformer
normalized_matrix = TfidfTransformer().fit_transform(bow_matrix)

In [0]:
# converting to graph
# This is a mirrored matrix, where both the rows and columns correspond to sentences, 
# and the elements describe how similar the two sentences are. Scores of 1 mean that the sentences are exactly the same, 
# while scores of 0 mean the sentences have no overlap.

similarity_graph = normalized_matrix * normalized_matrix.T
similarity_graph.toarray()

array([[1.        , 0.        , 0.13737879, 0.04767903, 0.04305016,
        0.04345599, 0.03330044, 0.05261648, 0.07798958, 0.        ,
        0.20047419],
       [0.        , 1.        , 0.0842143 , 0.07819597, 0.        ,
        0.05171612, 0.        , 0.        , 0.        , 0.        ,
        0.05807146],
       [0.13737879, 0.0842143 , 1.        , 0.        , 0.07004069,
        0.09648614, 0.1069042 , 0.06701793, 0.09437203, 0.20474295,
        0.1197599 ],
       [0.04767903, 0.07819597, 0.        , 1.        , 0.07558987,
        0.18678911, 0.05853972, 0.09249592, 0.10892262, 0.09110741,
        0.24159019],
       [0.04305016, 0.        , 0.07004069, 0.07558987, 1.        ,
        0.07055583, 0.02370685, 0.07272032, 0.17253418, 0.08262451,
        0.17789849],
       [0.04345599, 0.05171612, 0.09648614, 0.18678911, 0.07055583,
        1.        , 0.12952649, 0.06859301, 0.06837492, 0.13015945,
        0.15423071],
       [0.03330044, 0.        , 0.1069042 , 0.05853972, 0.

In [0]:
# Pagerank
import networkx as nx
nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
scores = nx.pagerank(nx_graph)
scores

{0: 0.08385008597047353,
 1: 0.07528144170324103,
 2: 0.09838560290878211,
 3: 0.09744270669005967,
 4: 0.0892656646614313,
 5: 0.09825615495072643,
 6: 0.0826112264425111,
 7: 0.08271845324598913,
 8: 0.09433617163509286,
 9: 0.08636822789376782,
 10: 0.11148426389792515}

In [0]:
ranked = sorted(((scores[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0][0])
ranked[0][1]

0.11148426389792515


'And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory.'

In [0]:
### Combine all of this into a function ###

import networkx as nx
import numpy as np
 
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
 
def textrank(document):
    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)
 
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

textrank(document)

[(0.11148426389792515,
  'And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory.'),
 (0.09838560290878211,
  'In his eyes she eclipses and predominates the whole of her sex.'),
 (0.09825615495072643,
  'He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position.'),
 (0.09744270669005967,
  'It was not that he felt any emotion akin to love for Irene Adler.'),
 (0.09433617163509286,
  'But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results.'),
 (0.0892656646614313,
  'All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind.'),
 (0.08636822789376782,
  'Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not